## Model Grafting

In [1]:
try:
    import google.colab
    IN_COLAB = True
    # %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    renderer = "colab"
except:
    IN_COLAB = False
    from IPython import get_ipython
    %load_ext autoreload
    %autoreload 2
    renderer = "jupyterlab"


In [2]:
%%bash
cd ../
pip install poetry
poetry install
cd notebooks

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached platformdirs-3.10.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.8.0
    Uninstalling platformdirs-3.8.0:
      Successfully uninstalled platformdirs-3.8.0
Installing dependencies from lock file

Package operations: 0 installs, 1 update, 0 removals

  • Downgrading platformdirs (3.10.0 -> 3.8.0)

Installing the current project: transformer-lens (0.0.0)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.167.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 5.2.0 which is incompatible.
sagemaker 2.167.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.23.3 which is incompatible.


In [4]:
!pip uninstall numpy -y

In [5]:
!pip install 'numpy<1.24'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 22.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.167.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 5.2.0 which is incompatible.
sagemaker 2.167.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.23.3 which is incompatible.
sparkmagic 0.20.5 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.


In [6]:

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = renderer


# Import stuff
import types
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import requests
import json

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import ast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib.cm import ScalarMappable
import pickle
if IN_COLAB: 
    import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens.utilities import devices
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Matplotlib is building the font cache; this may take a moment.


In [7]:
import torch.nn.functional as F
import torch.optim as optim


In [8]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def cuda():
    return torch.cuda.is_available()

def get_device(): 
    return "cuda" if cuda() else "cpu"

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

# Function to load a pickle object from a file
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_json(data, file_path):
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)

def load_json(file_path):
    with open(file_path, "r") as json_file:
        loaded_data = json.load(json_file)
    return loaded_data

device = get_device()
device

'cuda'

## Model

- fold_ln: Whether to fold in the LayerNorm weights to the subsequent linear layer. This does not change the computation.
- center_writing_weights: Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNormthis doesn't change the computation.
- center_unembed : Whether to center W_U (ie set mean to be zero). Softmax is translation invariant so this doesn't affect log probs or loss, but does change logits. Defaults to True.
- refactor_factored_attn_matrices: Whether to convert the factoredmatrices (W_Q & W_K, and W_O & W_V) to be "even". 

In [9]:
# MODEL_NAME = "gpt2-large"
MODEL_NAME = "gpt2-large"

model = HookedTransformer.from_pretrained(
        MODEL_NAME,
        center_unembed=True,  
        center_writing_weights=True,              # Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNorm this doesn't change the computation.      
        fold_ln=True,                             # Whether to  fold in the LayerNorm weights to the subsequent linear layer.
        refactor_factored_attn_matrices=True,
    )

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


## Dataset
We use the CounterFact Dataset. We subsample facts which the model can accurately predict. 

In [10]:
dataset = load_json("data/fact_dataset.json")


def sample_dataset(dataset, idx = None): 
    if idx is None: 
        prompt = "The {} is located in"
        subject = "Eiffel Tower"
        target = "Paris"
    else: 
        sample = dataset[idx]
        prompt = sample["requested_rewrite"]["prompt"]
        subject = sample["requested_rewrite"]["subject"]
        target = sample["requested_rewrite"]["target_true"]["str"]
    return prompt, subject, target


## Ablation Methods: 
- Noise ablation 
- Resample ablation

In [11]:
def noise_ablation(prompt, subject, target, n_noise_samples=5, vx=3):
    subject_tokens = model.to_tokens(subject)
    
    #shape: batch, n_tokens, embedding_dim
    subject_embedding = model.embed(subject_tokens)
    _, n_tokens, embedding_dim = subject_embedding.shape
    
    #noise: N(0,v), v = 3*std(embedding)
    embedding = model.W_E
    v = vx*torch.std(embedding, dim=0) #for each v in V
    noise = torch.randn(
        (n_noise_samples, n_tokens, embedding_dim)
    ).to(device) + v
    
    subject_embedding_w_noise = subject_embedding + noise
    
    #shape: batch, n_tokens, vocab_size (logits)
    unembedded_subject = model.unembed(subject_embedding_w_noise)

    noisy_subject_tokens = torch.argmax(unembedded_subject, dim=-1)
    noisy_subject_str = [
        model.to_string(nst) for nst in noisy_subject_tokens
    ]
    true_prompt = prompt.format(subject)
    corrupted_prompts = [
        prompt.format(nss) for nss in noisy_subject_str
    ]
    return true_prompt, corrupted_prompts, target

def resample_ablation(prompt, subject, target, n_noise_samples=20):
    subject_tokens = model.to_tokens(subject)
    embedding = model.W_E
    #we select n random rows from the embedding matrix
    permutations = torch.randperm(embedding.size(0))[:n_noise_samples]
    random_samples = embedding[permutations]
    #unsqueeze a token dimension between batch and embedding dims
    random_samples = random_samples.unsqueeze(dim=1)
    #we de-embed these rows
    random_embeddings = model.unembed(random_samples)
    random_tokens = torch.argmax(random_embeddings, dim=-1)
    random_subject_str = [
        model.to_string(t) for t in random_tokens
    ]
    corrupted_facts = [
        prompt.format(s) for s in random_subject_str
    ]
    true_fact = prompt.format(subject)
    return true_fact, corrupted_facts, target
    

    
    

In [12]:
def pad_from_left(tokens : torch.tensor, maxlen:int):
    pad_token = model.tokenizer.pad_token_id
    padded_tokenized_inputs = torch.zeros(tokens.shape[0], maxlen)
    
    n_pads = maxlen - tokens.shape[-1]
    padded_tokenized_inputs[:,n_pads] = pad_token
    padded_tokenized_inputs[:,n_pads:] = tokens
    return padded_tokenized_inputs.long()

def pad_to_same_length(clean_tokens, corrupted_tokens): 
    
    maxlen = max([clean_tokens.shape[-1], corrupted_tokens.shape[-1]])
    
    if clean_tokens.shape[-1] > corrupted_tokens.shape[-1]: 
        corrupted_tokens = pad_from_left(corrupted_tokens, maxlen)
    elif clean_tokens.shape[-1] < corrupted_tokens.shape[-1]: 
        clean_tokens = pad_from_left(clean_tokens, maxlen)
    return clean_tokens, corrupted_tokens

## Total Effect vs Direct Effect

In [24]:
def unembedding_function(residual_stack, cache, mlp=False) -> float:
    #we are only interested in applying the layer norm of the final layer on the final token
    #shape: [74, 5, 10, 1280] = n_layers, prompts, tokens, d_model
    z = cache.apply_ln_to_stack(residual_stack, layer = -1, mlp_input=mlp)
    z = z @ model.W_U
    return z

def patch(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    cache):
    corrupted_residual_component[:, pos, :] = cache[hook.name][:, pos, :]
    """
    Restore a patch in clean run. 
    """
    return corrupted_residual_component

def direct_effect(clean_cache,
                  corrupted_cache,
                  mle_token_idx,
                  target_token_idx, 
                  return_labels=False): 
    
    if return_labels: 
        residual_clean_stack, labels = clean_cache.decompose_resid(layer=-1, mode="all", return_labels=True)
    else: 
        residual_clean_stack = clean_cache.decompose_resid(layer=-1, mode="all", return_labels=False)
        
    residual_corrupted_stack = corrupted_cache.decompose_resid(layer=-1, mode="all", return_labels=False)
    
    #shape: [74, 4, 9, 50257] = n_layers, batch, tokens, vocab_size
    residual_clean_logits = unembedding_function(residual_clean_stack, clean_cache)
    residual_clean_logits = residual_clean_logits[:,:,-1,:]
    
    residual_corrupted_logits = unembedding_function(residual_corrupted_stack, corrupted_cache)
    residual_corrupted_logits = residual_corrupted_logits[:,:,-1,:] #get for final token only
    
    target_idx_expanded = target_token_idx.repeat(residual_clean_logits.shape[0],1,1)
    mle_idx_expanded = mle_token_idx.repeat(residual_clean_logits.shape[0],1,1)
    
    target_residual_corrupted_logit = residual_corrupted_logits.gather(index=target_idx_expanded, dim=-1)
    mle_residual_corrupted_logit = residual_corrupted_logits.gather(index=mle_idx_expanded, dim=-1)
    target_residual_clean_logit = residual_clean_logits.gather(index=target_idx_expanded, dim=-1)
    mle_residual_clean_logit = residual_clean_logits.gather(index=mle_idx_expanded, dim=-1)
    
    target_direct_effect = target_residual_corrupted_logit - target_residual_clean_logit
    mle_direct_effect = mle_residual_corrupted_logit - mle_residual_clean_logit #shape: 1, batch, target_tokens (1)
    
    #shapes: layers, batch, 1
    if return_labels: 
        return labels, mle_direct_effect, target_direct_effect
    
    return mle_direct_effect, target_direct_effect
    
    
def run_direct_vs_total_effect(
     clean_prompt: str,
     corrupted_prompts: List[str],
     target: str
    ):
    
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
    target_token_idx = model.to_tokens(target)[:,1]
    clean_tokens, corrupted_tokens = pad_to_same_length(clean_tokens, corrupted_tokens)
    clean_tokens = clean_tokens.expand(corrupted_tokens.shape[0], -1)
    target_token_idx = target_token_idx.expand(corrupted_tokens.shape[0], -1)

    clean_logits, clean_cache = model.run_with_cache(clean_tokens, return_type="logits")
    corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
    mle_token_idx = torch.argmax(clean_logits[:,-1,:], dim=-1).unsqueeze(-1)

    
    layer_names, mle_direct_effect, target_direct_effect = direct_effect(clean_cache,
                                                                          corrupted_cache,
                                                                          mle_token_idx,
                                                                          target_token_idx, 
                                                                          return_labels=False)
    

    print(layer_names)
    raise
    
    
  
    layer_names = []
    for i in range(n_layers): 
        patch_name = f"blocks.{i}.resid_post"
        
        hook_fn = partial(patch, pos=tok_pos, cache=clean_cache)
        hook = (patch_name, hook_fn)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[hook],
            return_type="logits"
        )   
        patched_logits = patched_logits[:,-1,:]
        patched_logits = patched_logits - patched_logits.mean(dim=-1, keepdim=True)
        patched_loss = F.softmax(patched_logits, dim=-1)

        causal_positions_logit_diff[i, tok_pos] = (patched_logits[:,target_idx] - corrupted_logits[:,target_idx]).to("cpu")
        causal_positions_prob_diff[i, tok_pos] = (patched_logits[:,target_idx] - corrupted_prob[:,target_idx]).to("cpu")
        causal_positions_KL_loss[i, tok_pos] = F.kl_div(patched_logits.log(), corrupted_prob, reduction='batchmean').to("cpu")
        causal_positions_CE_loss_diff[i, tok_pos] = F.cross_entropy(patched_logits, corrupted_logits).to("cpu")

    return {
        "causal_positions_logit_diff": causal_positions_logit_diff.to("cpu"), 
        "causal_positions_prob_diff": causal_positions_prob_diff.to("cpu"),
        "causal_positions_KL_loss": causal_positions_KL_loss.to("cpu"),
        "causal_positions_CE_loss_diff": causal_positions_CE_loss_diff.to("cpu"),
    }

In [25]:
clean_prompt, subject, target = sample_dataset(dataset)
original_fact, corrupted_facts, target = resample_ablation(clean_prompt, subject, target, n_noise_samples=10)

graft = run_direct_vs_total_effect(original_fact, corrupted_facts, target)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.75 GiB total capacity; 13.46 GiB already allocated; 10.81 MiB free; 13.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF